In [1]:
:!pwd

/projects/biproductions


: 1

In [2]:
import ClassyPrelude
import BiProductionsLib
import qualified Data.Vector as V
import qualified Data.Map as Map

: 1

In [3]:
--          "  The quick brown   fox jumped  over   the  lazy dog's  back ."
--wordId     00000 00001 00010 00011  00100 00101 00110 00111 01000 01001
--modifierId 00011 00011 00011 00011  01001 00100 01001 01000 01001 00011
--analogy    Analogy 0 00011 01001 00100

s1OMList =  [    3,   3,     3,    3,     9,    4,    9,    8,    9,    3] :: [Int]
s1ModifiersList = map (\(x,y) -> Modifier x y) $ zip ([0..] :: [Int]) s1OMList
s1Tenses = V.singleton (NTPast, NTPlusDelta)
s1Modifiers = V.fromList s1ModifiersList
s1Analogies = V.singleton (Analogy 0 3 9 4)
s1 = Sentence s1Tenses s1Modifiers s1Analogies
--bits = fmap V.fromList $ encodeOneHotOutput defaultOneHotCaps s1
--encodeOneHotOutput defaultOneHotCaps s1
--fmap (getBitGroups defaultOneHotCaps) bits
s1

Sentence [(NTPast,NTPlusDelta)] [Modifier 0 3,Modifier 1 3,Modifier 2 3,Modifier 3 3,Modifier 4 9,Modifier 5 4,Modifier 6 9,Modifier 7 8,Modifier 8 9,Modifier 9 3] [Analogy 0 3 9 4]


: 1

In [16]:
aList = [("hi",1),("george",2),("believer",3)] :: [(Text,Int)]
tshow $ Map.fromList aList 

"fromList [(\"believer\",3),(\"george\",2),(\"hi\",1)]"


: 1

In [5]:
--          " Jack   ate   the twinkie     ;  then    he   got  sick     ."
--wordId     00000 00001 00010   00011 00100 00101 00110 00111 01000 01001
--modifierId 00000 00011 00011   00000     - 01000 00110 01000 00110     -
--analogy    Analogy 0 00000 00011 00001, Analogy 1 00110 01000 00111 

s2OMList =  [    0,   3,     3,    0,    -1,    8,    6,    8,    6,    -1] :: [Int]
s2ModifiersList = map (\(x,y) -> Modifier x y) $ zip ([0..] :: [Int]) s2OMList
f2ModifiersList = filter (\(Modifier _ y) -> y >= 0) s2ModifiersList
s2Tenses = V.fromList [(NTPast, NTPlusDelta), (NTPlusDelta, NTPlusDelta)]
s2Modifiers = V.fromList f2ModifiersList
s2Analogies = V.fromList [(Analogy 0 0 3 1), (Analogy 1 6 8 7)]
s2 = Sentence s2Tenses s2Modifiers s2Analogies
--bits = fmap V.fromList $ encodeOneHotOutput defaultOneHotCaps s1
--encodeOneHotOutput defaultOneHotCaps s1
--fmap (getBitGroups defaultOneHotCaps) bits
s2

Sentence [(NTPast,NTPlusDelta),(NTPlusDelta,NTPlusDelta)] [Modifier 0 0,Modifier 1 3,Modifier 2 3,Modifier 3 0,Modifier 5 8,Modifier 6 6,Modifier 7 8,Modifier 8 6] [Analogy 0 0 3 1,Analogy 1 6 8 7]


: 1

In [6]:
decodeEncodeOneHotOutput defaultOneHotCaps s1
--bits = fmap V.fromList $ encodeOneHotOutput defaultOneHotCaps s2
--fmap (getBitGroups defaultOneHotCaps) bits
decodeEncodeOneHotOutput defaultOneHotCaps s2

Right (Sentence [(NTPast,NTPlusDelta)] [Modifier 0 3,Modifier 1 3,Modifier 2 3,Modifier 3 3,Modifier 4 9,Modifier 5 4,Modifier 6 9,Modifier 7 8,Modifier 8 9,Modifier 9 3] [Analogy 0 3 9 4],True)
Right (Sentence [(NTPast,NTPlusDelta),(NTPlusDelta,NTPlusDelta)] [Modifier 0 0,Modifier 1 3,Modifier 2 3,Modifier 3 0,Modifier 5 8,Modifier 6 6,Modifier 7 8,Modifier 8 6] [Analogy 0 0 3 1,Analogy 1 6 8 7],True)


: 1

In [7]:
ords = wordOrdinals ["killing", "trashing", "started", "killed", "bleed", "dragoned", "passer", "Blade"]
ords "Killed"
padded = padOrdinals (65536 :: Int) (256 :: Int) ords
padded "Killed"

([1,0,0],[1,0])
([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0])


: 1

In [8]:
-- Since we are going for comprehension, it seems unnecessary to digest sentences longer 
-- than 30 words. A compound sentence has 2 tenses, so cap tenses at 2. See:
-- https://insidegovuk.blog.gov.uk/2014/08/04/sentence-length-why-25-words-is-our-limit/
caps = defaultOneHotCaps
show caps

-- Here are the relevant types for the Linearizer:
--type BasicPhrase = ([Int],[Int],Int,[Int])
--type LinearPhrase a = [(Int,a)]
--data PhraseCode = SubjectCode Int BasicPhrase 
--                | ConjunctionCode BasicPhrase 
--                | AnalogyObjectCode Int Int BasicPhrase BasicPhrase Tense
--type Linearizer a = PhraseCode -> PhraseContext (LinearPhrase a)
--type WordGenerator a = LinearPhrase a -> PhraseContext (LinearPhrase Text)

-- The PhraseContext is essentially a random number generator:
--do getRandomN (low,high)
--do getRandom

maxModifiers = 5 :: Int

"OneHotCaps {ohcWords = 64, ohcTenses = 2}"


: 1

In [9]:
basicExampleData (nounExampleInfo False) (words "quick brown sick happy gnarly old") "fox" False 0 [False,True,True,False,False]
basicExampleData (verbExampleInfo False) (words "quickly happily nicely fairly recently") "kills" False 0 [False,True,True,False,False]
basicExampleData (verbFutureExampleInfo False) (words "quickly happily nicely fairly recently") "kill" False 0 [False,True,True,False,False]
basicExampleData (byVerbExampleInfo False) (words "quickly happily nicely fairly recently") "killing" False 0 [False,False,True,False,False]
exampleDataWithDescriptor [1,1] nounExampleInfo (words "quick brown sick happy gnarly old") (words "fox box salt") [False,False] False
exampleDataWithDescriptor [1,1,3] nounExampleInfo (words "quick brown sick happy gnarly old") (words "fox box salt") [False,False,True,False,False] False
exampleDataWithDescriptor [3,2] verbExampleInfo (words "quickly happily nicely fairly recently") (words "kills epically finally") [True,False,False,False,False] False
exampleDataWithDescriptor [3,2] byVerbExampleInfo (words "quickly happily nicely fairly recently") (words "jumping epically finally") [False,False,False,False,False] False

Just ([Modifier 0,Modifier 1,Modifier 2,Modifier 3,Modifier 4,MainWord],["the","very","very","happy","gnarly","fox"])
Just ([Modifier 0,Modifier 1,Modifier 2,Modifier 3,Modifier 4,MainWord],["quickly","very","very","fairly","recently","kills"])
Just ([Modifier 0,Modifier 1,Modifier 2,Modifier 3,Modifier 4,MainWord],["will","very","very","fairly","recently","kill"])
Just ([Modifier 0,Modifier 1,Modifier 2,Modifier 3,Modifier 4,MainWord],["by","happily","very","fairly","recently","killing"])
Just ([Modifier 0,MainWord,Other 0,Modifier 1],["the","fox","of","Box"])
Just ([Modifier 0,MainWord,Other 0,Modifier 4,Other 0,Modifier 1,Modifier 2,Modifier 3],["the","fox","of","Box","of","very","happy","Salt"])
Just ([Modifier 0,Modifier 1,Modifier 2,MainWord,Modifier 3,Modifier 4],["very","happily","nicely","kills","fairly","epically"])
Just ([Modifier 0,Modifier 1,Modifier 2,MainWord,Modifier 3,Modifier 4],["by","happily","nicely","jumping","fairly","epically"])


: 1

In [11]:
writeFileExamples "samples.txt" "wordLists.txt"

: 1